In [3]:
import numpy as np
from nltk.corpus import gutenberg
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [18]:
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print(chars)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 375542
['\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ']
total chars: 50


In [17]:
print(text[:2000])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on?
you sir, what trade are you?
  cobl. truely sir, in respect of a fine workman, i am
but as you would say, a cobler

   mur. but what trade art thou? answer me directly

   cob. a trade sir, that i hope i may vse, with a safe
conscience, which is indeed sir, a mender of bad soules

   fla. what trade thou knaue? thou naughty knaue,
what trade?
  cobl. nay i beseech you sir, be not out with me: yet
if you be out sir, i can mend you

   mur. what mean'st thou by that? mend mee, thou
sawcy fellow?

In [6]:
"""
Assemble a training set
"""

"""
Ignore sentence (and line) boundaries so the character-based model will learn
when to halt a sentence with a period ('.') or linefeed character ('\n').
"""
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 125168


In [7]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [8]:
# Build Model

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [11]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6450      
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________


In [12]:
epochs=6
batch_size=128

In [13]:
model_structure = model.to_json()
with open("shakespeare_lstm_model.json", "w") as json_file:
    json_file.write(model_structure)

for i in range(5):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=epochs)

    model.save_weights("shakes_lstm_weights_{}.h5".format(i + 1))
    print('Model saved.')

Epoch 1/6
125168/125168 [==============================] - 47s 372us/step - loss: 2.0473
Epoch 2/6
125168/125168 [==============================] - 45s 360us/step - loss: 1.6851
Epoch 3/6
125168/125168 [==============================] - 47s 374us/step - loss: 1.5794
Epoch 4/6
125168/125168 [==============================] - 47s 375us/step - loss: 1.5175
Epoch 5/6
125168/125168 [==============================] - 46s 365us/step - loss: 1.4742
Epoch 6/6
125168/125168 [==============================] - 48s 387us/step - loss: 1.4447
Model saved.
Epoch 1/6
125168/125168 [==============================] - 51s 411us/step - loss: 1.4187
Epoch 2/6
125168/125168 [==============================] - 48s 385us/step - loss: 1.3984
Epoch 3/6
125168/125168 [==============================] - 46s 370us/step - loss: 1.3800
Epoch 4/6
125168/125168 [==============================] - 46s 371us/step - loss: 1.3663
Epoch 5/6
125168/125168 [==============================] - 46s 365us/step - loss: 1.3549
Epoch 6/